In [1]:
import pandas as pd
import os
import simpledbf
from simpledbf import Dbf5

# Capacity

### Year Infos
le but c'est de créer une lecture automatisé de chaque année, donc on crée un dictionnaire avec le chemin et les noms de
colomnes d'interêt

In [4]:
year_infos = {}
for folder in os.listdir('Data_Treatment/Data/Capacity/'):
    if folder[:6] != 'eia860': #il y a un documents qui je ne sais pas d'où il vient, mais qui ne nous intêret pas
        continue
    year = int(folder[-4:])
    for file in os.listdir(f'Data_Treatment/Data/Capacity/{folder}'):
        
        if year < 2004: #objectif = 2001 ->2003. On doit les traîter separemment puisqu'ils sont un fichier dbf
            if file[:3].upper() == 'GEN' and file[-3:]=='dbf': #c'est le fichier desire, mais il est encore en format dbf
                    old_path = 'Data_Treatment/Data/Capacity/'+folder+'/'+file
                    dbf = Dbf5(old_path)
                    new_path = 'Data_Treatment/Data/Capacity/'+folder+'/'+file[:-3]+'csv'
                    dbf.to_csv(new_path)
                    path = new_path
                    energy_source = 'ENSOURCE1'
                    capacity = 'NAMEPLATE'
                    year_infos[year] = [path,energy_source,capacity]

        
        elif year<= 2012: #il y a eu un changement dans comment les fichiers s'appelent
            if file[:3].upper() == 'GEN':
                if file[-3:] not in ['xls','lsx']:
                    continue # je vais traiter les fichiers .dbf (2001-2003) après
                path = 'Data_Treatment/Data/Capacity/'+folder+'/'+file
                energy_source = 'ENERGY_SOURCE_1'
                capacity = 'NAMEPLATE'
                if year == 2012:
                    energy_source = 'Energy Source 1'
                    capacity = 'Nameplate Capacity (MW)'
                
                year_infos[year] = [path,energy_source,capacity]
        else:
            if file[:4] == '3_1_':
                path = 'Data_Treatment/Data/Capacity/'+folder+'/'+file
                energy_source = 'Energy Source 1'
                capacity = 'Nameplate Capacity (MW)'
                year_infos[year] = [path,energy_source,capacity]

NameError: name 'os' is not defined

## Codes et Technologies
le but c'est de bien separer les types de technologie à partir des codes existentes

In [3]:
#path_2020, energy_source_2020, capacity_2020= year_infos[2020][0],year_infos[2020][1], year_infos[2020][2]
#df_2020 = pd.read_excel(path_2020,header = 1)

#energies = ['Onshore Wind Turbine','Natural Gas Steam Turbine', 'Conventional Steam Coal', 'Solar Photovoltaic']
#technologies = {}
#codes = []

#for technology in df_2020['Technology'].unique():
#    if technology in energies: 
#        energy_source = df_2020['Energy Source 1'].where(df_2020['Technology']==technology).dropna().unique()
#        print(technology,energy_source)
#        technologies[technology] = energy_source
#        codes.append(energy_source)

#print(technologies)
#print(codes)

In [4]:
codes = ['WND','NG','BIT','LIG','SUB','RC','WC','SUN']
codes_technologies = {'WND': 'Onshore Wind Turbine', 'NG': 'Natural Gas Steam Turbine', 'SUN': 'Solar Photovoltaic',
                     'BIT': 'Conventional Steam Coal', 'LIG': 'Conventional Steam Coal', 'SUB': 'Conventional Steam Coal',
                     'RC': 'Conventional Steam Coal', 'WC': 'Conventional Steam Coal'}

## Delete_other_headers
on en utilise pour qu'on fasse le traitement des archives 2001->2003

In [5]:
def delete_other_headers(df): #dans les dfs plus agés (2001-03), il y avait plus d'un entete (header), et les numéros parfois n'étaient pas tous en format float
    df_interest = pd.DataFrame()
    df_interest['ENSOURCE1'],df_interest['NAMEPLATE'] = df['ENSOURCE1'],df['NAMEPLATE']
    #on doit effacer les lignes qui sont entetes, c-a-d: on trouvera celles où le valuer dans 'NAMEPLATE' 
    #est aussi 'NAMEPLATE'
    index = 0
    indexes=[]
    for line in df['NAMEPLATE']:
        if line == "NAMEPLATE":
            indexes.append(index)
        index+=1
    df_interest = df_interest.drop(indexes,axis=0)
    df_interest['NAMEPLATE'] = df_interest['NAMEPLATE'].apply(lambda x: float(x))
    #On a pu le faire directment puisque toutes les lignes étaient en format '1231.2', s'il avait quelqu'une differente,
    #il aurait jeté une exception
    return(df_interest)

# Merge Production & Capacity

In [6]:
energy_infos = {'solar':['Data_Treatment/treated_data/Production/solar_treated.csv','Solar Photovoltaic'],
                      'wind':['Data_Treatment/treated_data/Production/wind_treated.csv','Onshore Wind Turbine'],
                      'coal':['Data_Treatment/treated_data/Production/coal_treated.csv','Conventional Steam Coal'],
                     'gas':['Data_Treatment/treated_data/Production/gas_treated.csv','Natural Gas Steam Turbine']}

# Panel 

Dictionnaire pour agilizer la découverte des noms des colomns d'interêt

In [7]:
#year_columns = {}

#for year in year_infos:
#    path = year_infos[year][0]

#    if year <=2003:
        #df_year_raw = pd.read_csv(path,header = 0)

#    elif year<=2010:
        #df_year_raw = pd.read_excel(path,header = 0)
    
#    else: 
        #df_year_raw = pd.read_excel(path,header = 1)
    #year_columns[year] = df_year_raw.columns

avec la recherche, on peut faire le filtrage

In [8]:
for year in year_infos:
    if year>2003 and year<=2006:
        state = 'STATE'
        plant = 'PLNTCODE'
        month = 'INSVMONTH'
    elif year>2006 and year <=2008:
        state = 'STATE'
        plant = 'PLNTCODE'
        month = 'OPERATING_MONTH'
    elif year>2008 and year<=2011 :
        state = 'STATE'
        plant = 'PLANT_CODE'
        month = 'OPERATING_MONTH'
    elif year>2011:
        state = 'State'
        plant = 'Plant Code'
        month = 'Operating Month'
    year_infos[year].append(state)
    year_infos[year].append(plant)
    year_infos[year].append(month)

# Data Visualization

In [ ]:
def gen_sum_df(df,column):
    df_tool = df.groupby(['Year']).sum()
    df_result = pd.DataFrame()
    
    for i in range(len(df_tool)): # ici on utilise le range normal, 0 -> 2021-1973, puisque les ans presents dans df_renewable_month sont les indexes
        year = df_tool.iloc[i].name
        col = df_tool.iloc[i][column]
        new_row = pd.DataFrame({'Year': year, column: col}, index = [i])
        df_result=df_result.append(new_row, ignore_index = False)
    
    return df_result

def gen_count_df(df,column):
    df_tool = df.groupby(['Year']).count()
    df_result = pd.DataFrame()
    
    for i in range(len(df_tool)): # ici on utilise le range normal, 0 -> 2021-1973, puisque les ans presents dans df_renewable_month sont les indexes
        year = df_tool.iloc[i].name
        col = df_tool.iloc[i][column]
        new_row = pd.DataFrame({'Year': year, column: col}, index = [i])
        df_result=df_result.append(new_row, ignore_index = False)
    
    return df_result

def gen_max_df(df,column):
    df_tool = df.groupby(['Year']).max()
    df_result = pd.DataFrame()
    
    for i in range(len(df_tool)): # ici on utilise le range normal, 0 -> 2021-1973, puisque les ans presents dans df_renewable_month sont les indexes
        year = df_tool.iloc[i].name
        col = df_tool.iloc[i][column]
        new_row = pd.DataFrame({'Year': year, column: col}, index = [i])
        df_result=df_result.append(new_row, ignore_index = False)
    
    return df_result

def iterative_merge(list_dfs):
    df = list_dfs[0]
    for i in range(1,len(list_dfs)-1):
        df_merged = pd.merge(df,list_dfs[i+1], on = 'Year')
        df = df_merged
        df_merged = pd.DataFrame()
    return(df)

# Technology Filter

In [2]:
technology_code = {'Coal':['ANT','BIT','LIG','SUB','SGC','WC'],
                  'Petroleum Products': ['DFO','JF','KER','PC','PG','RFO','SG','WO'],
                   'Natural Gas':['NG'],
                   'Other Gases': ['BFG','OG'],
                   'Nuclear':['NUC'],
                   'Hidroeletric':['WAT'],
                   'Wood and Wood-Delivered Fuels':['WDS','WDL','BLW'],
                   'Other Biomass':['AB','MSW', 'OBG', 'OBL', 'OBS', 'LFG', 'SLW'],
                   'Solar':['SUN'],
                   'Wind':['WND'],
                   'Geothermical':['GEO'],
                   'Other':['PUR','WH','TDF','MWH']
                  }

code_to_technology = {}

for technology in technology_code:
    for code in technology_code[technology]:
        code_to_technology[code]=technology
        
        
def convert_to_technology(x):
    try:
        return(code_to_technology[x])
    except:
        return('Not in the Appendix')

# Map USA

In [2]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))